# Traffic Forecasting on Highways using Graph Neural Networks

simulateous time series forecasting for multiple (215) sensors on the Austrian highway\
Reserach Questions:
* how can GNNs be used to model and predict traffic states?
* in particular, how can the time dimension bei incorporated into GNNs? GNN+LSTM vs. Attention-GNN
* how do these methods compare to the cuurently used methods (XGBoost on individual sensors)?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as pe
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.renderers.default='iframe'
import sys
import os

%load_ext autoreload
%autoreload 2

sys.path.append('/data/sensorfusion/ARMS/philipp/')
import tools.tools_preprocessing as tools_preprocessing
from tools.tools_sensors import fetch_db_tables, get_sensor_metadata
from pipeline_bt_clustering import cluster_bt_tts

sys.path.append('/data/sensorfusion/ARMS/philipp/ML_project')
from encode_graph import create_adj_mat, plot_tls_graph, dataframe_to_node_values

In [2]:
file_path = "../../data/processed"
sensor_data_path = '../../data/sensor_pairs'
dataset_mode = 'VDE_A01_1_10mins'

In [3]:
tls = pd.read_parquet(f'{file_path}/tls_processed_from_2022-09-01_to_2023-03-19.parquet')
tls = tls.set_index('detectiontime_at_destination')

In [4]:
tls_sensor_metadata = get_sensor_metadata(tls, mode = "tls")
tls_sensor_metadata['direction'] = tls_sensor_metadata['direction'].astype(int)
tls_sensor_metadata

,source_detector_road,source_detector_km,direction,detector_id,count,X,Y
0,A01,18.23,1,"MQ_A01_1_026,833",535993,16.119926,48.176036
1,A01,18.23,2,"MQ_A01_2_026,833",558307,16.119926,48.176036
2,A01,29.21,1,"MQ_A01_1_037,810",783935,15.986449,48.154321
3,A01,29.21,2,"MQ_A01_2_037,810",789324,15.986449,48.154321
4,A01,35.70,1,"MQ_A01_1_044,300",717697,15.902359,48.153768
...,...,...,...,...,...,...,...
210,A23,13.96,2,"MQ_A23_2_013,955",739650,16.447177,48.215558
211,A23,14.23,1,"MQ_A23_1_014,229",774718,16.447151,48.217992
212,A23,14.56,2,"MQ_A23_2_014,558",746842,16.447388,48.220946
213,A23,16.33,1,"MQ_A23_1_016,330",757405,16.456536,48.235328


In [5]:
tls_sensor_metadata.groupby('source_detector_road').count()

,source_detector_km,direction,detector_id,count,X,Y
source_detector_road,,,,,,
A01,117,117,117,117,117,117
A07,62,62,62,62,62,62
A23,36,36,36,36,36,36


Dataset Experiments:
* road graphs: A23 (36 nodes), A01 (117 nodes), all (215 nodes)
* directions: 1, 2 and both
* time horizons: 60 mins, 48 ts -> 2 days; 30 mins, 48 ts -> 1 day; 5 mins, 48 ts -> 4h

In [6]:
lookup_sensor_in_raw_metadata = lambda sensor_name: tls_sensor_metadata[tls_sensor_metadata.detector_id == sensor_name].index[0] 

In [7]:
lookup_sensor_in_raw_metadata('MQ_A23_2_013,955')

210

In [8]:
tls_sensor_metadata.sort_values(['direction', 'source_detector_road', 'source_detector_km']).reset_index()

,index,source_detector_road,source_detector_km,direction,detector_id,count,X,Y
0,0,A01,18.23,1,"MQ_A01_1_026,833",535993,16.119926,48.176036
1,2,A01,29.21,1,"MQ_A01_1_037,810",783935,15.986449,48.154321
2,4,A01,35.70,1,"MQ_A01_1_044,300",717697,15.902359,48.153768
3,6,A01,41.69,1,"MQ_A01_1_050,285",721040,15.836737,48.176264
4,8,A01,47.90,1,"MQ_A01_1_056,500",723985,15.756348,48.185718
...,...,...,...,...,...,...,...,...
210,206,A23,12.30,2,"MQ_A23_2_012,300",708779,16.433039,48.204287
211,208,A23,12.90,2,"MQ_A23_2_012,900",757648,16.438690,48.208154
212,210,A23,13.96,2,"MQ_A23_2_013,955",739650,16.447177,48.215558
213,212,A23,14.56,2,"MQ_A23_2_014,558",746842,16.447388,48.220946


## encode the graph of the sensor network

we **create an adjacency matrix** based on the distance betweenn two nodes and the density of the sensor network:\
for every sensor: the function uses a sliding window approach to calculate the distance between neighboring sensor locations, prunes the window based on density, and creates edges in the adjacency matrix accordingly

In [135]:
config = {'roads': [['A23'], ['A01'], ['A23', 'A01', 'A07']],
         'directions': [[1], [2], [1,2]],
         'time_horizons': ['60min', '30min', '5min']}

config = {'roads': [['A01']],
         'directions': [[1]],
         'time_horizons': ['60min']}


clean_str = lambda s: str(s).replace("[", "").replace("]", "").replace("'", "").replace(' ','')


graphs_settings = pd.DataFrame()

for road in config['roads']:
    for direction in config['directions']:
        for time_horizon in config['time_horizons']:
            graph_name = f"TLS_{clean_str(road)}_{clean_str(direction)}_{time_horizon}"
            directory = f"data/{graph_name}"
            if not os.path.exists(directory):
                os.makedirs(directory)
                
            ## GRAPH GENERATION
            tls_metadata = tls_sensor_metadata[(tls_sensor_metadata.source_detector_road.isin(road)) & (tls_sensor_metadata.direction.isin(direction))]
            adj_mat, nr_edges = create_adj_mat(tls_metadata, output_path_vde = directory)
            if time_horizon == '60min':
                plot_tls_graph(adj_mat, tls_metadata, clean_str(road), clean_str(direction), graph_name)
            single_graph = pd.Series({'name': graph_name, 'road': clean_str(road), 'direction': clean_str(direction),'nr_sensors': len(tls_metadata),'nr_edges': nr_edges})

            ## FILTERING & RESAMPLING
            tls_1 = tls[tls.detector_id.isin(tls_metadata.detector_id)]
            tls_1 = tls_1[(tls_1.VehicleType == 'motorVehicle')]
            tls_1 = tls_1.groupby(['detector_id'])[['TrafficVolumeInLastMinute', 'Velocity']].resample(time_horizon).mean().reset_index(['detector_id','detectiontime_at_destination'])

            ## CREATE TRAINING DATASET
            data, sensors, timestamps = dataframe_to_node_values(tls_1, directory)
            single_graph['nr_timestamps'] = len(timestamps)

            graphs_settings = pd.concat([graphs_settings, single_graph.to_frame().T])
        
        

364 edges created


data missing for 39 timestamps


In [136]:
graphs_settings

,name,road,direction,nr_sensors,nr_edges,nr_timestamps
0,TLS_A01_1_60min,A01,1,59,364,4681


In [188]:
graphs_settings.to_csv('graphs_settings.csv', index=False)

## Visulisation

currently not distinguishing between cars & lorries

In [163]:
tls_1 = tls[tls.detector_id.isin(tls_metadata.detector_id)]
tls_1 = tls_1[(tls_1.VehicleType == 'motorVehicle')]

In [167]:
tls_1 = tls_1.groupby(['detector_id'])[['TrafficVolumeInLastMinute', 'Velocity']].resample('30min').mean().reset_index(['detector_id','detectiontime_at_destination'])

In [169]:
tls_1

,detector_id,detectiontime_at_destination,TrafficVolumeInLastMinute,Velocity
0,"MQ_A01_1_026,833",2022-09-05 00:00:00,1.958333,87.250000
1,"MQ_A01_1_026,833",2022-09-05 00:30:00,1.916667,84.916667
2,"MQ_A01_1_026,833",2022-09-05 01:00:00,1.909091,82.727273
3,"MQ_A01_1_026,833",2022-09-05 01:30:00,1.500000,83.000000
4,"MQ_A01_1_026,833",2022-09-05 02:00:00,1.363636,77.909091
...,...,...,...,...
1977738,"MQ_S10_2_016,842",2023-03-18 21:30:00,1.842105,99.421053
1977739,"MQ_S10_2_016,842",2023-03-18 22:00:00,1.809524,102.809524
1977740,"MQ_S10_2_016,842",2023-03-18 22:30:00,2.047619,98.666667
1977741,"MQ_S10_2_016,842",2023-03-18 23:00:00,2.260870,98.739130


In [172]:
start_dt = "2023-03-01 00:00:00"
end_dt = "2023-03-02 00:00:00"

df = tls_1[(tls_1.detectiontime_at_destination > start_dt) & (tls_1.detectiontime_at_destination < end_dt)]
df = df.merge(tls_metadata, how='inner', on="detector_id").dropna()
# df["abnormal"] = df.Velocity.apply(lambda x: 1 if x > 90 else 0)
df['detectiontime_at_destination'] = df['detectiontime_at_destination'].astype("category")

fig = pe.scatter(
    df, x="X", y="Y",
    animation_frame="detectiontime_at_destination",
    animation_group="detector_id",
    size="TrafficVolumeInLastMinute", 
    color="Velocity",
    color_continuous_scale=[[0, 'blue'], [1, 'red']],
    hover_name="detector_id",
    height=900
)

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 200

fig.show()

## Dataset Creation

In [173]:
data, sensors, timestamps = dataframe_to_node_values(tls_1, f"data/{dataset_mode}")
data.shape

data missing for 83 timestamps


(9361, 215, 2)

In [178]:
len(timestamps)

9361

we use our custom dataset class: VDEDatasetLoader \
adapted the implementation from the METR_LA dataset: https://pytorch-geometric-temporal.readthedocs.io/en/latest/_modules/torch_geometric_temporal/dataset/metr_la.html \
`_generate_task()` takes the static graph at all timestamps (here: 28k aggregated to 10min intervals)\
generates time series samples by sliding a window over the timestamps (here: context = 144 timestamps (24h) for predicting 144 timestamps into the future)\
`get_dataset()` returns a `StaticGraphTemporalSignal(edges, edge_weights, features, targets)` from the Pytorch Geometric Temporal library\
note: tragets for time step $t$ are features for time step $t+1$

## Modeling

base class: `GNNLightningModule(pl.LightningModule)`, PL is wrapper for Pytorch having `training_step(), validation_step(), saving and loading the best model, learning rate reducer, pyperparameter tuner, wandb,...`\
input shape: (batch_size, num_nodes, num_features, num_timesteps) -> e.g. (10, 106, 2, 144)\
plug in different models in `pl.Trainer` call:

### 2 Pytroch Geometric GCNConv layer + LSTM:
(first GCNConv: `(batch_size * num_nodes, num_timesteps, num_features) -> (batch_size * num_nodes, num_timesteps, hidden_dim)` => `(1060, 144, 2) -> (1060, 144, 32)`\
2nd GCNConv and LSTM continue using these shapes\
`hidden_dim = 32, num_layers_lstm = 3` \
batch_size: 10 \
min val loss: 0.114 \
time per epoch: 00:42 min


### Attention Network: A3TGCN2:
https://arxiv.org/abs/2006.11583 \
`hidden_dim = 32` \
batch_size: 40 \
min val loss: 0.380 \
time per epoch: 03:48 min


![title](A3T-GCN.png)

note: both models trained on the same GPU, 8GB RAM


In [206]:
# np.save(f"data/{dataset_mode}/sensors.npy", sensors)
# np.save(f"data/{dataset_mode}/timestamps.npy", timestamps)
sensors, timestamps = np.load(f"data/{dataset_mode}/sensors.npy", allow_pickle=True), np.load(f"data/{dataset_mode}/timestamps.npy", allow_pickle=True)

In [ ]:
torchaudio torchmetrics torchvision

In [ ]:
pytorch-lightning pytorch-mutex pytorch_scatter pytorch_sparse 

if 3D extract is needed (for pkw, lkw, etc.): https://stackoverflow.com/questions/59546746/pivot-a-pandas-dataframe-into-a-3d-numpy-array

## Running the models

### Evaluation (from file)

In [45]:
graph_settings_results = pd.read_csv('results/graph_settings_results.csv')
graph_settings_results.sort_values('val_loss')

,name,road,direction,nr_sensors,nr_edges,nr_timestamps,net_name,hidden_dim,val_loss,running_time,nr_train_samples,nr_test_samples,batch_size,nr_epochs,best_model_size
6,TLS_A23_1_30min,A23,1,18,124,9361,GCN_LSTM,100,0.233556,245.605120,7032,1759,256,26,2780511
18,TLS_A23_2_30min,A23,2,18,124,9361,GCN_LSTM,100,0.255877,231.722913,7032,1759,256,23,2780511
30,"TLS_A23_1,2_30min",A23,"1,2",36,248,9361,GCN_LSTM,100,0.267276,504.596698,7032,1759,128,31,2782943
14,TLS_A23_2_60min,A23,2,18,124,4681,GCN_LSTM,100,0.285369,208.014542,3409,853,256,31,2780511
2,TLS_A23_1_60min,A23,1,18,124,4681,GCN_LSTM,100,0.290184,240.464388,3409,853,256,40,2780511
0,TLS_A23_1_60min,A23,1,18,124,4681,GCN_LSTM,50,0.297151,217.345174,3409,853,512,57,722655
26,"TLS_A23_1,2_60min",A23,"1,2",36,248,4681,GCN_LSTM,100,0.301587,417.079837,3409,853,128,44,2782943
24,"TLS_A23_1,2_60min",A23,"1,2",36,248,4681,GCN_LSTM,50,0.317912,458.422478,3409,853,256,88,725087
8,TLS_A23_1_5min,A23,1,18,124,56161,GCN_LSTM,50,0.456955,345.390381,43867,10967,512,24,722655
7,TLS_A23_1_30min,A23,1,18,124,9361,A3TGCN,100,0.517924,2217.418977,7032,1759,1024,279,1164091


,timestamp,type,sensor,value,batch,score
0,-48,x,"MQ_A23_2_000,377",10.049999,19,flow
1,-47,x,"MQ_A23_2_000,377",5.583334,19,flow
2,-46,x,"MQ_A23_2_000,377",3.327274,19,flow
3,-45,x,"MQ_A23_2_000,377",2.351852,19,flow
4,-44,x,"MQ_A23_2_000,377",1.795918,19,flow
...,...,...,...,...,...,...
2587,43,y_pred,"MQ_A23_2_016,330",73.779550,41,velocity
2588,44,y_pred,"MQ_A23_2_016,330",75.169720,41,velocity
2589,45,y_pred,"MQ_A23_2_016,330",73.838460,41,velocity
2590,46,y_pred,"MQ_A23_2_016,330",73.761604,41,velocity


In [101]:
# reslut_file_name="outputs/GCN_LSTM_TLS_A23_1_30min_100.csv"
reslut_file_name="outputs/GCN_LSTM_TLS_A23_2_60min_100.csv"

eval_dfs = pd.read_csv(reslut_file_name, index_col=0)
eval_dfs = eval_dfs.pivot(index=['sensor', 'type', 'timestamp', 'batch'], columns='score', values='value').reset_index()
eval_dfs['flow'] = eval_dfs['flow'] - eval_dfs['flow'].min()

In [102]:
eval_dfs

score,sensor,type,timestamp,batch,flow,velocity
0,"MQ_A23_2_000,377",x,-48,13,36.845006,75.183334
1,"MQ_A23_2_000,377",x,-48,25,5.068592,77.792450
2,"MQ_A23_2_000,377",x,-48,68,5.995007,76.666664
3,"MQ_A23_2_000,377",x,-48,79,28.528340,72.883330
4,"MQ_A23_2_000,377",x,-48,99,33.178341,72.716670
...,...,...,...,...,...,...
12955,"MQ_A23_2_016,330",y_pred,47,13,20.774906,67.542595
12956,"MQ_A23_2_016,330",y_pred,47,25,3.447266,78.607960
12957,"MQ_A23_2_016,330",y_pred,47,68,10.721491,78.160190
12958,"MQ_A23_2_016,330",y_pred,47,79,20.392929,73.362236


In [103]:
batch_ids = eval_dfs.batch.drop_duplicates().sort_values().values
batch_ids

array([13, 25, 68, 79, 99])

In [104]:
eval_dfs_one = eval_dfs[eval_dfs.batch == batch_ids[1]]
fig = pe.line(eval_dfs_one, x='timestamp', y='flow', facet_col="sensor", facet_col_wrap=2, color="type", facet_row_spacing = 0.01, height = 1300, width = 700,
             title = "GCN_LSTM on A23 direction 2 - 60mins - hidden dim = 100")

fig.show()
fig.write_image("first_results.png")


In [214]:
df_eval = eval_dfs_one.merge(tls_metadata_1, how='inner', left_on="sensor", right_on="detector_id").dropna()
df_eval["sensor"] = df_eval["sensor"] + ' - ' + df_eval['type']
df_eval['X'] = df_eval.apply(lambda x: x['X'] + 0.0 if x['type'] == 'y_pred' else x['X'], axis = 1)
df_eval['Y'] = df_eval.apply(lambda x: x['Y'] + 0.1 if x['type'] == 'y_pred' else x['Y'], axis = 1)
# df['detectiontime_at_destination'] = df['detectiontime_at_destination'].astype("category")

fig = pe.scatter(
    df_eval[df_eval['type'] != "x"],
    x="X", y="Y",
    animation_frame="timestamp",
    animation_group="sensor",
    size="flow", 
    color="velocity",
    color_continuous_scale=[[0, 'blue'], [1, 'red']],
    hover_name="sensor",
    height=900
)

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 100

fig.show()

## Next Steps
* comapre to baseline (ASFiNAG) - train 200+ regressors (for every sensor)
* in-depth-analysis w.r.t. performance, prediction timescales (minutes to hours), computational cost, storage costs
* 2 models for both directions vs. 1 unified model for all sensors? assumption: model will interpolate between both directions due to invariance learned by convolution





baselines: data from yesterday, 
incorporating additional inforation
remove sensors and inference - experiment

TODO:
    Baseline Modell - Radim kontaktieren 
    Vergelich Performance, Computational Cost, Prediction Duration für alle Sensoren
    
JE nachdem, ob's geht:
vlt 2 Modelle für beide Richtungen nehmen
annahme: Modell wird über beide Richtungen interpolieren
einfach DIstributions danach anschauen (z.b. tage-weise): ob sie sich für beide Richtungen unterscheiden
invarianz gelernt von Convolution---


### Contrete next steps:
* fine tuning
* two models for both direction vs 1 unified model

    

## Master Thesis Topics:

Traffic states & travel times:
* extend modeling of the trafic data with content messages -> knowledge graph representation, digital twin

Trajoectories:
* framework for "language" of trajecotries - describing anomaly

Video:
* fingerprint of individual cars from video (webcams) for traveltimes (Sigi's suggestion)
* learning homography for applying 3D view to localized bird's eye view for all cameras

## Evaluation of hyperparameter tuning

In [130]:
import json
import pandas as pd

# Open the file in read mode
with open("results/results_GNN_LSTM.json", "r") as file:
    # Load the JSON data from the file
    json_data = json.load(file)

# Create a list to store the rows
rows = []

# Iterate over each JSON element in the array
for json_element in json_data:
    # Add the JSON element as a row to the list
    rows.append(json_element)

# Create a DataFrame from the list of rows
df = pd.DataFrame(rows)
df = df.sort_values('loss')

# Display the DataFrame
df
corrs = df.drop(columns=['run_name']).corr().round(2)
corrs = corrs[['loss', 'run_time']].T

fig = pe.imshow(corrs, text_auto=True, color_continuous_scale='RdBu_r', width = 700, height = 340,
               title = "correlation of hyperparameter")
fig.show()
fig.write_image("corr.png")



In [133]:
import shutil

# Specify the folder path and output zip file path
folder_path = 'results'
output_path = 'results'

# Zip the folder
shutil.make_archive(output_path, 'zip', folder_path)


'/data/sensorfusion/ARMS/philipp/ML_Project/results.zip'

In [131]:
pwd

'/data/sensorfusion/ARMS/philipp/ML_Project'